In [ ]:
import numpy as np
import pandas as pd
import re
import html
import ast

# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords

# nltk.download('stopwords')
# nltk.download('punkt')

## Read data

In [ ]:
data = pd.read_csv('concatenatedAll.csv')

In [ ]:
data

In [ ]:
#important_hashtags = ['probirth','prolife','proabortion','antiabortion','prochoice']

In [ ]:
def process_tweet(text):
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = html.unescape(text)
    text = re.sub('[^A-Za-z ]+', '', text)
    text = text.strip().lower()
    
    stop_words = stopwords.words('english')
    word_tokens = word_tokenize(text)
    word_tokens = [w for w in word_tokens if (not w in stop_words) and (len(w)>1) and (len(w)<25)]
    text = ' '.join(word_tokens)
    
    return text

In [ ]:
data['text'] = data['text'].apply(process_tweet)

In [ ]:
data

In [ ]:
p_data = data[data.text.str.len() > 0]

In [ ]:
p_data[['text','id','author_id','Class']].to_csv('PreprocessedAll.csv',index=False)

In [ ]:
#temptweets.drop(['correct_author_id','Class'],axis=1).to_csv('preprocessed_tweets_correct.csv',index=False)

In [ ]:
temptweets=pd.read_csv('PreprocessedAll.csv')
tempusers=pd.read_csv('ConcatenatedUsers_m3_eth.csv')
tempusers.rename(columns={'id':'author_id'},inplace=True)
tempusers.drop_duplicates('author_id',inplace=True)
tempusers['author_id']=tempusers['author_id'].astype(str)
temptweets['author_id']=temptweets['author_id'].astype(str)
tempusers['NewClass'] = 1

In [ ]:
f = open('classDict.txt','r')
classDict = ast.literal_eval(f.readlines()[0])

In [ ]:
len(set(temptweets.author_id)-set(tempusers.author_id))

In [ ]:
m =pd.merge(temptweets,tempusers,on='author_id')

In [ ]:
m['opinion'] = m.author_id.map(classDict)
m.opinion.value_counts()

In [ ]:
temptweets

In [ ]:
correctMapping={}

for badId in list(diff):
    temp=badId[:6]
    try:
        userrow=tempusers[tempusers.author_id.str.startswith(temp)].iloc[0]
        correctMapping[badId]=userrow.author_id
    except:
        pass

In [ ]:
str(temptweets.iloc[0].correct_author_id)

In [ ]:
temptweets['correct_author_id']=temptweets.author_id.map(correctMapping)

In [ ]:
for i,row in temptweets.iterrows():
    if str(row['correct_author_id'])!='nan':
        temptweets.at[i,'author_id']=row['correct_author_id']

In [ ]:
tempUsers = pd.read_csv('concatenated_users_labelled.csv')

In [ ]:
def isinTexas(x):
    x=str(x)
    x=x.lower()
    
    tx=['texas','tx','dallas','houston','san antonio','austin','fort worth','el paso']
    for i in tx:
        if i in x:
            return True
    return False

In [ ]:
tempUsers['texas'] = tempUsers['location'].apply(isinTexas)

In [ ]:
tempUsers[tempUsers.texas==1]

In [ ]:
tempUsers.texas.value_counts()

In [ ]:
tempUsers.to_csv('withtexas.csv',index=False)